In [1]:
import pandas as pd
import spacy
import re
import tldextract
import tensorflow as tf
import numpy as np
import unicodedata

nlp = spacy.load('en_core_web_sm')

STOP_WORDS = nlp.Defaults.stop_words

nRowsRead = None  # specify 'None' if want to read whole file
postStrings = []
typeStrings = []


2022-09-07 11:19:05.334648: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-07 11:19:05.334672: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-07 11:19:06.868642: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-07 11:19:06.868662: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-07 11:19:06.868687: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (marko-pc): /proc/driver/nvidia/version does not exist


In [2]:
df = pd.read_csv('./input/mbti_1.csv', delimiter=',', nrows=nRowsRead)
df.dataframeName = 'mbti_1.csv'

In [3]:
# making array of array of posts
for person,type in zip(df['posts'],df['type']):
    postStrings.append([type,person.split('|||')])

# removing excess single parentheses

for i in range(0,len(postStrings)):
    postStrings[i][1][0] = postStrings[i][1][0][1:]
    postStrings[i][1][-1] = postStrings[i][1][-1][:-1]

In [4]:
#splitting posts into list of posts
df['posts'] = df['posts'].apply(lambda x: x.split('|||'))

# removing excess single parentheses
def removeParentheses(posts):
    posts[0] = posts[0][1:]
    posts[-1] = posts[-1][:-1]
    return posts
df['posts'] = df['posts'].apply(lambda x: removeParentheses(x))

df.head()

,type,posts
0,INFJ,"[http://www.youtube.com/watch?v=qsXHcwe3krw, h..."
1,ENTP,[I'm finding the lack of me in these posts ver...
2,INTP,[Good one _____ https://www.youtube.com/wat...
3,INTJ,"[Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,"[You're fired., That's another silly misconcep..."


In [5]:
#label encoding type

def labelEncodeType(type):
  if type == 'ISTJ' or type == 'ISFJ' or type == 'ESTJ' or type == 'ESFJ':
    type = 0 #GUARDIAN
  elif type == 'ISTP' or type == 'ISFP' or type == 'ESTP' or type == 'ESFP':
    type = 1 #ARTISAN
  elif type == 'INFJ' or type == 'INFP' or type == 'ENFP' or type == 'ENFJ':
    type = 2  # IDEALIST
  elif type == 'INTJ' or type == 'INTP' or type == 'ENTP' or type == 'ENTJ':
    type = 3  # RATIONALIST
  return type

df['type'] = df['type'].apply(lambda x: labelEncodeType(x))
df.head()

,type,posts
0,2,"[http://www.youtube.com/watch?v=qsXHcwe3krw, h..."
1,3,[I'm finding the lack of me in these posts ver...
2,3,[Good one _____ https://www.youtube.com/wat...
3,3,"[Dear INTP, I enjoyed our conversation the o..."
4,3,"[You're fired., That's another silly misconcep..."


In [6]:
df['type'].value_counts()

2    4167
3    3311
1     745
0     452
Name: type, dtype: int64

In [7]:
URL_REGEX = r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))"""
DOMAIN_REGEX = "^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?([^:\/?\n]+)"
COMMENT_REGEX = r"\[.*?\]"
ELLIPSIS_REGEX = r"^(\ *\.{3}\ *)|(\ *\.{3}\ *)$"

def urlReplace(stringToReplace):
	domain = tldextract.extract(stringToReplace.group())
	return domain.domain + '.' + domain.suffix


**CONSIDER TRYING WITHOUT STOPWORD REMOVAL**

In [8]:
def containsOnlyUrlsOrNumbers(post):
	splits = post.split()
	for str in splits:
		if(not re.search(URL_REGEX,str) and not re.search('^\d+$',str) and not re.search('^\.\.\.+$',str)):
			return 1
	return 0

def containsComments(post):
	splits = post.split('] ')
	for str in splits:
		if(not re.search(COMMENT_REGEX,str)):
			return 1
		return 0

def removeEllipsis(post):
	return re.sub(ELLIPSIS_REGEX,'',post)

def removeWhitespace(post):
	post = " ".join(post.split())
	return post

def removeStopWords(post):
	post = " ".join([t for t in post.split() if t not in STOP_WORDS])
	return post

def removeSpecialChars(post): #ALSO REMOVES PUNCTUATION
	post = re.sub('[^A-Z a-z 0-9-]+','',post)
	return post

def removeAccentedChars(post):
	post = unicodedata.normalize('NFKD',post).encode('ascii','ignore').decode('utf-8','ignore')
	return post

def formatPerson(person):
	person = list(map(lambda x: removeWhitespace(x), person))
	person = [post for post in person if (
		containsOnlyUrlsOrNumbers(post) and containsComments(post))]
	person = list(map(lambda x: removeEllipsis(x), person))
	person = list(map(lambda x: re.sub(URL_REGEX, urlReplace, x), person))
	person = list(map(lambda x: removeStopWords(x),person))
	person = list(map(lambda x: removeSpecialChars(x), person))
	person = list(map(lambda x: removeAccentedChars(x), person))
	return person

df['posts']= df['posts'].apply(lambda x:formatPerson(x))

df.head()

,type,posts
0,2,[enfp intj moments youtubecom sportscenter pla...
1,3,"[Im finding lack posts alarming, Sex boring it..."
2,3,"[Good youtubecom, Of course I I know thats bl..."
3,3,[Dear INTP I enjoyed conversation day Esoteric...
4,3,"[Youre fired, Thats silly misconception That a..."


In [9]:
#do the nlp tokenization with spaCy

def convertToLemmas(posts):
  docs = nlp.pipe(posts, n_process=-1, disable=["tok2vec", "parser"])
  lemmaList = []
  postList = []
  for doc in docs:
    for token in doc:
      lemma = str(token.lemma_)
      if lemma == '-PRON-' or lemma == 'be':
        lemma = token.text
      lemmaList.append(lemma)
    postList.append(" ".join(lemmaList))
    lemmaList= []
  return postList

#for the dataframe:
df['posts']= df['posts'].apply(lambda x:convertToLemmas(x))
df.head()

,type,posts
0,2,[enfp intj moments youtubecom sportscenter pla...
1,3,"[I m find lack post alarm, sex bore its positi..."
2,3,"[good youtubecom, of course I I know that s ..."
3,3,[dear INTP I enjoy conversation day Esoteric g...
4,3,"[you re fire, that s silly misconception that ..."


In [10]:
df.to_csv(path_or_buf="./csv_files/df.csv")
# df = pd.read_csv(filepath_or_buffer="./csv_files/df.csv")

In [9]:
#SPLITTING POSTS INTO SEPERATE ROWS
df = df.explode('posts')
df

,type,posts
0,2,enfp intj moments youtubecom sportscenter play...
0,2,What life-changing experience life
0,2,youtubecom youtubecom On repeat today
0,2,May PerC Experience immerse you
0,2,The thing INFJ friend posted facebook committi...
...,...,...
8674,2,I going close facebook months back wanting abl...
8674,2,30 Seconds Mars - All collections It fitting m...
8674,2,I seen it agree I actually think time I watche...
8674,2,Ok watched Underworld 4 Awakening good film Co...


In [10]:
df['type'].value_counts()

2    197249
3    155773
1     34619
0     21308
Name: type, dtype: int64

In [11]:
#common words removal
freq_comm = pd.Series(' '.join(df['posts'].tolist()).split()).value_counts()
f20= freq_comm[:20]

In [12]:
# df['posts'] = df['posts'].apply(lambda x: ' '.join([t for t in x.split() if t not in f20]))

In [13]:
#rare words removal
rare= freq_comm[freq_comm.values==1]
# rare= freq_comm[freq_comm.values<3]

In [14]:
df['posts'] = df['posts'].apply(lambda x: ' '.join([t for t in x.split() if t not in rare]))

In [15]:
df0 = df[df['type'] == 0].sample(10000)
df1 = df[df['type'] == 1].sample(10000)
df2 = df[df['type'] == 2].sample(10000)
df3 = df[df['type'] == 3].sample(10000)

dfr = pd.concat([df0,df1,df2,df3],)
dfr

,type,posts
1705,0,Im quiet person its nature There times topic b...
630,0,I sense makes sense
6014,0,Would travel If so where And long Yes Either d...
7938,0,ESTP In MBTI its ok pick I E I think Se domina...
6742,0,The - The American Midland
...,...,...
4847,3,Same Objects bodies homework friendsyou
1671,3,This actually difficult question Social life l...
5447,3,Ok I wont pretend I know this How exactly INTP...
677,3,fall love male enfp fall love female entp inst...


In [16]:
y = dfr['type']

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
cv = CountVectorizer(dtype='b')
text_counts = cv.fit_transform(dfr['posts'])

In [19]:
text_counts.toarray().shape

(40000, 27807)

In [20]:
dfr_bow = pd.DataFrame(text_counts.toarray(),columns=cv.get_feature_names_out())

In [21]:
dfr_bow.head(2)

,00,000,001,002,005,006,00s,018,0200,038,...,zoquaro,zosio913,zster,zubz,zuchinni,zuckerberg,zuko,zynthaxx,zyzz,zz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# dfr_bow.to_csv(path_or_buf="./csv_files/dfr_bow.csv")
# dfr.to_csv(path_or_buf="./csv_files/dfr.csv")

## ML ALGORITHMS

In [23]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [24]:
rfc = RandomForestClassifier(random_state=42,n_estimators=500,n_jobs=-1,max_depth=8)

In [25]:
%%time

def classify(X,y):
  scaler = MinMaxScaler(feature_range=(0,1))
  X=scaler.fit_transform(X)

  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size= 0.2,random_state=42,stratify=y)

  rfc.fit(X_train,y_train)
  y_pred = rfc.predict(X_test)
  ac = accuracy_score(y_test,y_pred)
  print("accuracy: ",ac)


classify(dfr_bow,y)

accuracy:  0.35875
CPU times: user 8min 54s, sys: 1min 23s, total: 10min 18s
Wall time: 2min 32s
